# dataloaders

> Bundle trn/val/test datasets together, and add helpful functionality for viewing data

In [ ]:
#| hide
#|default_exp dataloaders

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from matplotlib import pyplot as plt

from fastcore.all import *
import torch
from torch import nn
from torch import Tensor
from datasets import load_dataset
from torch.utils.data import DataLoader
import pandas as pd 
import numpy as np
from datasets import Dataset

In [ ]:
#| export
def random_splitter(dataset, valid_pct=0.2):
    n = len(dataset)
    _val_idxs = np.random.rand(n)<valid_pct
    return dataset.select(mask2idxs(~_val_idxs)), dataset.select(mask2idxs(_val_idxs))

In [ ]:
#| export
def load_fashion_mnist(sample=None):
    name = "fashion_mnist"
    ds = load_dataset(name)
    train,test = ds['train'],ds['test']
    if sample is not None: ds['train'] = ds['train'].select(range(sample))
    x,y = ds['train'].features.keys()
    trn, val = random_splitter(train)
    
    @inplace
    def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
    
    def create_dl(ds, shuffle=True, batch_size=64):
        _ds = ds.with_transform(transformi)
        return DataLoader(_ds, batch_size=batch_size, shuffle=shuffle,)

    return map(create_dl, [trn,val,test])

In [ ]:
#| export
class DataLoaders():
    def __init__(self, train, valid, test=None): store_attr()

    def show_batch(self, ds='train', rows=3, cols=3):
        ds = getattr(self, ds)
        b = first(ds)
        cn = ds.dataset.features['label'].names

        fig,ax = plt.subplots(rows,cols,figsize=(5,5))
        for row in range(rows):
            for col in range(cols):
                cell = row * cols + col
                ax[row,col].imshow(b['image'][cell].reshape(28,28),cmap='gray')
                ci = int(b['label'][cell])
                ax[row,col].set_title(f"{int(b['label'][cell])} : {cn[ci]}")
                ax[row,col].axis('off')


In [ ]:
dls = DataLoaders(*load_fashion_mnist())
dls.show_batch()

In [ ]:
class SimpleNet(nn.Module):
    '''simple net'''
    def __init__(self,in_features,hidden_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features,hidden_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_features,out_features)

    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
model = SimpleNet(784,64,10)
test_eq(model(first(dls.train)[x]).shape,(64,10))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()